## 邊界抽取 Boundary Extraction
公式中 A 為你打算做Boundary Extraction 的目標(Source Image)，B 為你設定的Structures Element。
簡單來說就是用 A 跟 B 做 Erosion 會得到一個比較小的A，再用原來的A 減掉比較小的A，那麼多出來的地方就剛好是邊界了。

In [1]:
import cv2
import numpy as np

img = cv2.imread('./img/j.png',0) 
kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(img,kernel,iterations = 1)

subtracted = cv2.subtract(img,erosion)


In [2]:
cv2.imshow('Input', img)
cv2.imshow('Result', erosion)
cv2.imshow("Subtracted", subtracted)
cv2.waitKey(0)

-1